In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_list_recommend = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/List_Hotel_Recommend.csv')

In [4]:
df_test = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/data_test.csv')
df_hotel = pd.read_csv('/content/drive/MyDrive/DS300_DoAn/data/data_hotel.csv')

In [5]:
# Tổng hợp IDhotel theo từng IDuser
id_hotels = df_test.groupby('IDuser')['IDhotel'].apply(list).reset_index()

In [6]:
# Tạo một từ điển ánh xạ tên khách sạn sang ID
hotel_name_to_id = {row["NameHotel"]: row["IDhotel"] for _, row in df_hotel.iterrows()}

# Chuyển đổi tên khách sạn trong dataframe "df_recommend" sang dạng ID
df_list_recommend["HotelID"] = df_list_recommend["List Hotel Recommended"].apply(lambda x: [hotel_name_to_id[name.strip("'")] for name in eval(x)])

In [7]:
def precision_at_k(actual, predicted, k):
    # Giới hạn danh sách dự đoán đến vị trí k
    predicted_k = predicted[:k]

    # Số lượng mục dự đoán chính xác
    correct_predictions = len(set(actual) & set(predicted_k))

    # Precision tại vị trí k
    precision = correct_predictions / k

    return precision

In [8]:
def average_precision_at_k(test_data, pred_data, k):
    users = set(test_data['IDuser']) & set(pred_data['IDuser'])
    num_users = len(users)
    total_precision = 0

    for user in users:
        user_actual = test_data.loc[test_data['IDuser'] == user, 'IDhotel'].tolist()[0]
        user_predicted = pred_data.loc[pred_data['IDuser'] == user, 'HotelID'].tolist()[0]

        user_precision = precision_at_k(user_actual, user_predicted, k)
        total_precision += user_precision

    average_precision = total_precision / num_users

    return average_precision

In [9]:
actual = id_hotels[['IDuser', 'IDhotel']]
predict = df_list_recommend[['IDuser','HotelID']]

In [15]:
average_precision = average_precision_at_k(actual, predict,5)
average_precision

0.125

In [11]:
def recall_at_k(actual, predicted, k):
    # Giới hạn danh sách dự đoán đến vị trí k
    predicted_k = predicted[:k]

    # Số lượng mục dự đoán chính xác
    correct_predictions = len(set(actual) & set(predicted_k))

    # Số lượng mục thực tế
    total_actual = len(actual)

    # Recall tại vị trí k
    recall = correct_predictions / total_actual

    return recall

def average_recall_at_k(test_data, pred_data, k):
    users = set(test_data['IDuser']) & set(pred_data['IDuser'])
    num_users = len(users)
    total_recall = 0

    for user in users:
        user_actual = test_data.loc[test_data['IDuser'] == user, 'IDhotel'].tolist()[0]
        user_predicted = pred_data.loc[pred_data['IDuser'] == user, 'HotelID'].tolist()[0]

        user_recall = recall_at_k(user_actual, user_predicted, k)
        total_recall += user_recall

    average_recall = total_recall / num_users

    return average_recall

In [16]:
average_recall = average_recall_at_k(actual, predict,5)
average_recall

0.02505801959626288